borrowed from bella's version by her suggestion

In [5]:
!pip install -q evaluate torchinfo bitsandbytes peft

In [6]:
import numpy as np
import pandas as pd
from datasets import Dataset
import evaluate
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from torchinfo import summary
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BitsAndBytesConfig
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Datasets, Tokenizer, and Model

In [9]:
train_df = pd.read_csv('/content/drive/MyDrive/266/MLMA_Toxicity_Detection_Deep_Learning/Data/combined_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/266/MLMA_Toxicity_Detection_Deep_Learning/Data/combined_val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/266/MLMA_Toxicity_Detection_Deep_Learning/Data/combined_test.csv')

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [10]:
# !rm -r "google/rembert" # if the next cell gives error "not a string", run this cell

In [11]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('google/rembert')
model = AutoModelForSequenceClassification.from_pretrained('google/rembert')
model.gradient_checkpointing_enable()

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

sentencepiece.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

Some weights of RemBertForSequenceClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def preprocess(batch):
    # Ensure all entries are strings and handle missing values
    review_texts = [str(x) for x in batch['text_cleaned']]
    encoded = tokenizer(
        review_texts,
        padding='max_length',
        truncation=True,
        max_length=64,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors="pt"
    )

    # Return input features and the label
    return {
        'input_ids': encoded['input_ids'],
        'token_type_ids': encoded['token_type_ids'],
        'attention_mask': encoded['attention_mask'],
        'labels': batch['hatespeech'] # Keep the 'hatespeech' column as labels
    }


train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)

# Identify columns to remove (all except the new ones)
original_columns = train_df.columns.tolist()
columns_to_keep = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
columns_to_remove = [col for col in original_columns if col not in columns_to_keep]

# Remove original columns from the datasets
train_dataset = train_dataset.remove_columns(columns_to_remove)
val_dataset = val_dataset.remove_columns(columns_to_remove)

Map:   0%|          | 0/30702 [00:00<?, ? examples/s]

Map:   0%|          | 0/3901 [00:00<?, ? examples/s]

In [13]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 30702
})

In [14]:
# Show a sample input and output

text = train_df.iloc[5000]['text_cleaned']
encoded_input = tokenizer(text, return_tensors='pt')

# Move input tensors to the same device as the model
device = model.device
encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

output = model(**encoded_input)

print(text)
print(encoded_input)
print(output)

هههه وانت لا تنسى ان تمسح بصقة اليهودي الذي إستباح ارضك وعِرضك من....... اما بول البعير ياجاهل فهو فخر ل…
{'input_ids': tensor([[   312,  20236,  36763,    717,  36367,   3698,  13916,  85378,   2459,
           6971, 112100, 113172,  32804,    573, 181206, 122607,  10624, 134299,
          93834,  17442,    573, 220069,   2262,  39649,   1332,  65420,   2262,
           1109,  54673,  14963,  69486,    978,  55184,   9292,  14900,  23464,
          58995,    573, 202668,   2598,  41749,   1687,    655,    313]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0222,  0.1256]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [15]:
stop

NameError: name 'stop' is not defined

## Model Structure

In [ ]:
for name, param in model.named_parameters():
    print(name)

In [ ]:
# Freeze all of the pre-trained BERT layers to make the fine tuning go much faster.
# Then later we'll try unfreezing some or all layers and see what works better.
# We need to keep the final classification layer unfrozen no matter what,
# because that's a new layer that hasn't been trained at all yet, and needs to be trained for our task.


for name, param in model.named_parameters():
    if name.split(".")[0] == "rembert":
        param.requires_grad = False

In [ ]:

# confirm all pre-trained layers are frozen
summary(model)

## Model Training

In [ ]:
batch_size = 256
num_epochs = 5

training_args = TrainingArguments(
    output_dir="model_checkpoints",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, # to help memory management
    num_train_epochs=num_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to='none',
    # og ^

    # KEY for logging:
    # logging_strategy="steps",         # or "epoch"
    # logging_steps=10,                 # log every 10 steps
    # report_to='none',                 # or 'tensorboard'/'all'
    # disable_tqdm=False,

    # label_names=['labels'] # try to fix no log issue (for loss)
    # no log might be fine for untrainable / frozen
)

In [ ]:
import evaluate
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Load the accuracy metric
accuracy_metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    # For single-label classification, use argmax to get predicted labels
    predicted_labels = np.argmax(predictions, axis=1)

    # Compute metrics
    accuracy = accuracy_metric.compute(predictions=predicted_labels, references=labels)
    precision = precision_score(labels, predicted_labels, zero_division=0)
    recall = recall_score(labels, predicted_labels, zero_division=0)
    f1 = f1_score(labels, predicted_labels, zero_division=0)

    # For AUC, we need the probabilities of the positive class (assuming binary classification)
    # If predictions are logits, apply softmax or sigmoid to get probabilities
    if predictions.shape[-1] == 2: # Check if it's a binary classification output
        probabilities = torch.softmax(torch.tensor(predictions), dim=1)[:, 1].numpy()
        auc = roc_auc_score(labels, probabilities)
    else: # Handle cases that are not binary classification
        auc = None # Or compute a different type of AUC if applicable

    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': auc
    }

In [ ]:
print(train_dataset[0])

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# trainer.can_return_loss = True # try to fix no log issue (for loss)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
stop

In [ ]:
# Get a sample from the validation dataset
sample_index = 2500
val_sample = val_dataset[sample_index]

# Prepare the input for the model
inputs = {
    'input_ids': torch.tensor([val_sample['input_ids']]),
    'attention_mask': torch.tensor([val_sample['attention_mask']]),
    'token_type_ids': torch.tensor([val_sample['token_type_ids']]) # Added token_type_ids
}

# Move input tensors to the same device as the model
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Get the model prediction
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**inputs)

# Access the correct logits from the dictionary output and calculate probabilities
# hatespeech_logits = outputs['hatespeech_logits'] # Removed incorrect key
logits = outputs['logits'] # Accessed logits using the correct key
hatespeech_probabilities = torch.softmax(logits, dim=1)
predicted_hatespeech_label = torch.argmax(logits, dim=1).item()

print(f"Text: {tokenizer.decode(val_sample['input_ids'])}")
print(f"True Hatespeech Label: {val_sample['labels']}") # Corrected label access
print(f"Predicted Hatespeech Label: {predicted_hatespeech_label}")
print(f"Hatespeech Probabilities: {hatespeech_probabilities.tolist()}")

In [ ]:
true_hatespeech_labels = []
predicted_hatespeech_labels = []

device = model.device

for val_sample in val_dataset:
    inputs = {
        'input_ids': torch.tensor([val_sample['input_ids']]).to(device),
        'attention_mask': torch.tensor([val_sample['attention_mask']]).to(device)
    }

    with torch.no_grad():
        outputs = model(**inputs)

    hatespeech_logits = outputs['logits']
    predicted_hatespeech_label = torch.argmax(hatespeech_logits, dim=1).item()

    true_hatespeech_labels.append(val_sample['labels'])
    predicted_hatespeech_labels.append(predicted_hatespeech_label)

In [ ]:
cm = confusion_matrix(true_hatespeech_labels, predicted_hatespeech_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Hatespeech Prediction')
plt.show()

# Unfrozen

In [ ]:
# Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained('google/rembert')
# model = AutoModelForSequenceClassification.from_pretrained('google/rembert')
# model.gradient_checkpointing_enable()

In [ ]:
# unfreezing
for name, param in model.named_parameters():
    if name.split(".")[0] == "rembert":
        param.requires_grad = True

In [ ]:
summary(model)

In [ ]:
batch_size = 256
num_epochs = 5

training_args = TrainingArguments(
    output_dir="model_checkpoints",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, # to help memory management
    num_train_epochs=num_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to='none',
    # og ^

    # try to fix no log issue (for loss)
    # KEY for logging:
    logging_strategy="steps",         # or "epoch"
    logging_steps=10,                 # log every 10 steps
    disable_tqdm=False,
)

In [ ]:
import evaluate
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Load the accuracy metric
accuracy_metric = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    # For single-label classification, use argmax to get predicted labels
    predicted_labels = np.argmax(predictions, axis=1)

    # Compute metrics
    accuracy = accuracy_metric.compute(predictions=predicted_labels, references=labels)
    precision = precision_score(labels, predicted_labels, zero_division=0)
    recall = recall_score(labels, predicted_labels, zero_division=0)
    f1 = f1_score(labels, predicted_labels, zero_division=0)

    # For AUC, we need the probabilities of the positive class (assuming binary classification)
    # If predictions are logits, apply softmax or sigmoid to get probabilities
    if predictions.shape[-1] == 2: # Check if it's a binary classification output
        probabilities = torch.softmax(torch.tensor(predictions), dim=1)[:, 1].numpy()
        auc = roc_auc_score(labels, probabilities)
    else: # Handle cases that are not binary classification
        auc = None # Or compute a different type of AUC if applicable

    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': auc
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.can_return_loss = True # try to fix no log issue (for loss)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

## Multi-Label Prediction

In [ ]:
stop

In [16]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

example = train_dataset[0]
example

{'text_cleaned': 'يلعن شرفك يا حكم!!!',
 'language': 'arabic',
 'directness_label': 2,
 'target_label': 2,
 'group_label': 12,
 'hatespeech': 0,
 'hatespeech.1': 0,
 'directness_label.1': 2,
 'target_label.1': 2,
 'group_label.1': 12,
 'sentiment_anger': 0,
 'sentiment_confusion': 0,
 'sentiment_disgust': 0,
 'sentiment_fear': 0,
 'sentiment_indifference': 0,
 'sentiment_normal': 1,
 'sentiment_sadness': 0,
 'sentiment_shock': 0,
 'language_encoded': 2}

In [17]:
from datasets import DatasetDict

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text_cleaned', 'language', 'directness_label', 'target_label', 'group_label', 'hatespeech', 'hatespeech.1', 'directness_label.1', 'target_label.1', 'group_label.1', 'sentiment_anger', 'sentiment_confusion', 'sentiment_disgust', 'sentiment_fear', 'sentiment_indifference', 'sentiment_normal', 'sentiment_sadness', 'sentiment_shock', 'language_encoded'],
        num_rows: 30702
    })
    validation: Dataset({
        features: ['text_cleaned', 'language', 'directness_label', 'target_label', 'group_label', 'hatespeech', 'hatespeech.1', 'directness_label.1', 'target_label.1', 'group_label.1', 'sentiment_anger', 'sentiment_confusion', 'sentiment_disgust', 'sentiment_fear', 'sentiment_indifference', 'sentiment_normal', 'sentiment_sadness', 'sentiment_shock', 'language_encoded'],
        num_rows: 3901
    })
    test: Dataset({
        features: ['text_cleaned', 'language', 'directness_label', 'target_label', 'group_label', 'hatespeech', 

In [18]:
example = dataset['train'][13000]
example

{'text_cleaned': "c'est un gros mongol quand même",
 'language': 'french',
 'directness_label': 0,
 'target_label': 3,
 'group_label': 8,
 'hatespeech': 1,
 'hatespeech.1': 1,
 'directness_label.1': 0,
 'target_label.1': 3,
 'group_label.1': 8,
 'sentiment_anger': 0,
 'sentiment_confusion': 0,
 'sentiment_disgust': 0,
 'sentiment_fear': 0,
 'sentiment_indifference': 0,
 'sentiment_normal': 0,
 'sentiment_sadness': 0,
 'sentiment_shock': 1,
 'language_encoded': 1}

In [19]:
# Original binary labels
binary_label_columns = [
    'hatespeech',
    'sentiment_indifference',
    'sentiment_disgust',
    'sentiment_sadness',
    'sentiment_anger',
    'sentiment_shock',
    'sentiment_normal',
    'sentiment_confusion',
    'sentiment_fear'
]

# Get unique values for multiclass labels to determine the range for one-hot encoding
directness_values = sorted(train_df['directness_label'].unique().tolist())
target_values = sorted(train_df['target_label'].unique().tolist())
group_values = sorted(train_df['group_label'].unique().tolist())

# Create names for one-hot encoded binary labels
directness_binary_labels = [f'directness_label_{val}' for val in directness_values]
target_binary_labels = [f'target_label_{val}' for val in target_values]
group_binary_labels = [f'group_label_{val}' for val in group_values]

# Combine all binary label names
all_binary_labels = binary_label_columns + directness_binary_labels + target_binary_labels + group_binary_labels

# Update id2label and label2id
id2label = {idx:label for idx, label in enumerate(all_binary_labels)}
label2id = {label:idx for idx, label in enumerate(all_binary_labels)}

# The total number of binary labels
num_binary_labels = len(all_binary_labels)

print(f"Total number of binary labels: {num_binary_labels}")
print(f"All binary labels: {all_binary_labels}")

Total number of binary labels: 36
All binary labels: ['hatespeech', 'sentiment_indifference', 'sentiment_disgust', 'sentiment_sadness', 'sentiment_anger', 'sentiment_shock', 'sentiment_normal', 'sentiment_confusion', 'sentiment_fear', 'directness_label_0', 'directness_label_1', 'directness_label_2', 'target_label_0', 'target_label_1', 'target_label_2', 'target_label_3', 'target_label_4', 'target_label_5', 'target_label_6', 'group_label_0', 'group_label_1', 'group_label_2', 'group_label_3', 'group_label_4', 'group_label_5', 'group_label_6', 'group_label_7', 'group_label_8', 'group_label_9', 'group_label_10', 'group_label_11', 'group_label_12', 'group_label_13', 'group_label_14', 'group_label_15', 'group_label_16']


In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('google/rembert')
model = AutoModelForSequenceClassification.from_pretrained('google/rembert',
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(all_binary_labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of RemBertForSequenceClassification were not initialized from the model checkpoint at google/rembert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import torch
import numpy as np

# Get unique values for multiclass labels to determine the range for one-hot encoding
directness_values = sorted(train_df['directness_label'].unique().tolist())
target_values = sorted(train_df['target_label'].unique().tolist())
group_values = sorted(train_df['group_label'].unique().tolist())

def preprocess_multioutput_data(examples):
  # take a batch of texts
  text = [str(x) for x in examples["text_cleaned"]]
  # encode them, returning PyTorch tensors directly
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=150, return_tensors="pt")

  # Create list of binary label tensors
  binary_labels_list = [
      torch.tensor(examples["hatespeech"]),
      torch.tensor(examples["sentiment_anger"]),
      torch.tensor(examples["sentiment_confusion"]),
      torch.tensor(examples["sentiment_disgust"]),
      torch.tensor(examples["sentiment_fear"]),
      torch.tensor(examples["sentiment_indifference"]),
      torch.tensor(examples["sentiment_normal"]),
      torch.tensor(examples["sentiment_sadness"]),
      torch.tensor(examples["sentiment_shock"])
  ]

  # Perform one-hot encoding for multiclass labels and add to the list
  for i, val in enumerate(directness_values):
      binary_labels_list.append(torch.tensor([1.0 if label == val else 0.0 for label in examples["directness_label"]]))

  for i, val in enumerate(target_values):
      binary_labels_list.append(torch.tensor([1.0 if label == val else 0.0 for label in examples["target_label"]]))

  for i, val in enumerate(group_values):
      binary_labels_list.append(torch.tensor([1.0 if label == val else 0.0 for label in examples["group_label"]]))


  # Stack all binary label tensors along dimension 1
  labels_tensor = torch.stack(binary_labels_list, dim=1).float()

  # Add the stacked labels tensor to the encoding dictionary
  encoding['labels'] = labels_tensor

  return encoding

In [22]:


# Apply the preprocessing function with batching
encoded_dataset = dataset.map(preprocess_multioutput_data, batched=True, remove_columns=dataset['train'].column_names)


Map:   0%|          | 0/30702 [00:00<?, ? examples/s]

Map:   0%|          | 0/3901 [00:00<?, ? examples/s]

Map:   0%|          | 0/3901 [00:00<?, ? examples/s]

In [23]:
example = encoded_dataset['train'][13000]
print(example.keys())

print(tokenizer.decode(example['input_ids']))

print(example)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
[CLS] c'est un gros mongol quand même[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]
{'input_ids': [312, 897, 584, 1121, 621, 33105, 65473, 14488, 4573, 313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
stop

NameError: name 'stop' is not defined

# Frozen Multi-Label

In [ ]:
for name, param in model.named_parameters():
    print(name)

In [ ]:
# Freeze all of the pre-trained BERT layers to make the fine tuning go much faster.
# Then later we'll try unfreezing some or all layers and see what works better.
# We need to keep the final classification layer unfrozen no matter what,
# because that's a new layer that hasn't been trained at all yet, and needs to be trained for our task.


for name, param in model.named_parameters():
    if name.split(".")[0] == "rembert":
        param.requires_grad = False

In [ ]:
summary(model)

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

batch_size = 64
metric_name = "eval_f1" # Changed to monitor a specific per-label F1

args = TrainingArguments(
    f"google/rembert",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, # to help memory management
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    # Use the micro-averaged F1 as the metric to monitor
    metric_for_best_model=metric_name,
    greater_is_better=True, # F1 score is better when higher
    report_to='none', # Disable reporting to wandb for this example

    # try to fix no log issue (for loss)
    # KEY for logging:
    logging_strategy="steps",         # or "epoch"
    logging_steps=10,                 # log every 10 steps
    disable_tqdm=False,
)

# Add EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3, # Number of evaluation epochs to wait before stopping
    early_stopping_threshold=0.0 # Minimum improvement to consider as progress
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, balanced_accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np # Import numpy

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels.astype(int) # Ensure labels are integers
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true.flatten(), y_pred.flatten())
    precision = precision_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    recall = recall_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'balanced_accuracy': balanced_accuracy,
               'precision': precision,
               'recall': recall}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions

    # Ensure labels are a 2D numpy array of integers
    labels_np = np.array(p.label_ids)
    if labels_np.ndim == 1:
        # This might happen if the batch size is 1, reshape to (1, num_labels)
        labels_np = labels_np.reshape(1, -1)
    elif labels_np.ndim > 2:
        # Handle cases where labels might have extra dimensions unexpectedly
        # This might require more specific reshaping based on the actual structure
        # For now, let's assume the expected shape is (batch_size, num_labels)
        # and try to flatten and reshape if necessary.
        # A safer approach might be to raise an error or log a warning here.
        print(f"Warning: Unexpected label dimension: {labels_np.ndim}")
        # Attempt to flatten and reshape - adjust if your data structure is different
        labels_np = labels_np.reshape(labels_np.shape[0], -1)


    result = multi_label_metrics(
        predictions=preds,
        labels=labels_np)
    return result

In [ ]:
#forward pass
# # Check the type of the data after setting the format
# print(f"Type of input_ids[0]: {encoded_dataset['train']['input_ids'][0].type()}")
# print(f"Type of labels[0]: {encoded_dataset['train'][0]['hatespeech_labels'].type()}")

# Also print a sample example to see the structure
print("Sample example after setting format:")
print(encoded_dataset['train'][0])

# Manually get a sample and prepare input for the model
sample = encoded_dataset['train'][0]

# Convert lists to PyTorch tensors before unsqueezing
input_ids_tensor = torch.tensor(sample['input_ids']).unsqueeze(0)
attention_mask_tensor = torch.tensor(sample['attention_mask']).unsqueeze(0)
labels_tensor = torch.tensor(sample['labels']).unsqueeze(0)

# Move tensors to the same device as the model
device = model.device
input_ids_tensor = input_ids_tensor.to(device)
attention_mask_tensor = attention_mask_tensor.to(device)
labels_tensor = labels_tensor.to(device)


outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor, labels=labels_tensor)
outputs

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.can_return_loss = True # try to fix no log issue (for loss)

In [ ]:
print(encoded_dataset['train']['labels'][13000])

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
logits = outputs.logits
logits.shape

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print("Predicted labels (names):", predicted_labels)
print("Predicted labels (binary values):", predictions)

In [ ]:
true_labels = []
predicted_logits = []

device = trainer.model.device

for sample in encoded_dataset["validation"]:
    inputs = {
        'input_ids': torch.tensor([sample['input_ids']]).to(device),
        'attention_mask': torch.tensor([sample['attention_mask']]).to(device)
    }

    with torch.no_grad():
        outputs = trainer.model(**inputs)

    # Append the true labels and predicted logits
    true_labels.append(sample['labels'])
    predicted_logits.append(outputs.logits.squeeze().cpu().numpy())


In [ ]:
# Convert lists to numpy arrays
true_labels_np = np.array(true_labels)
predicted_logits_np = np.array(predicted_logits)

# Define indices for each label group based on all_binary_labels
hatespeech_idx = all_binary_labels.index('hatespeech')
sentiment_indices = [all_binary_labels.index(label) for label in binary_label_columns if 'sentiment' in label]
directness_indices = [all_binary_labels.index(label) for label in directness_binary_labels]
target_indices = [all_binary_labels.index(label) for label in target_binary_labels]
group_indices = [all_binary_labels.index(label) for label in group_binary_labels]


# Split true labels and predicted logits
true_hatespeech = true_labels_np[:, hatespeech_idx]
predicted_hatespeech_logits = predicted_logits_np[:, hatespeech_idx]

true_sentiment_labels = true_labels_np[:, sentiment_indices]
predicted_sentiment_logits = predicted_logits_np[:, sentiment_indices]

true_directness_labels = true_labels_np[:, directness_indices]
predicted_directness_logits = predicted_logits_np[:, directness_indices]

true_target_labels = true_labels_np[:, target_indices]
predicted_target_logits = predicted_logits_np[:, target_indices]

true_group_labels = true_labels_np[:, group_indices]
predicted_group_logits = predicted_logits_np[:, group_indices]

print("True Hatespeech Labels shape:", true_hatespeech.shape)
print("Predicted Hatespeech Logits shape:", predicted_hatespeech_logits.shape)
print("True Sentiment Labels shape:", true_sentiment_labels.shape)
print("Predicted Sentiment Logits shape:", predicted_sentiment_logits.shape)
print("True Directness Labels shape:", true_directness_labels.shape)
print("Predicted Directness Logits shape:", predicted_directness_logits.shape)
print("True Target Labels shape:", true_target_labels.shape)
print("Predicted Target Logits shape:", predicted_target_logits.shape)
print("True Group Labels shape:", true_group_labels.shape)
print("Predicted Group Logits shape:", predicted_group_logits.shape)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_metrics_and_confusion_matrix(true_labels, predicted_logits, threshold=0.5, is_multilabel=True):
    """
    Calculates classification metrics and confusion matrix.

    Args:
        true_labels (np.ndarray): True labels (binary).
        predicted_logits (np.ndarray): Predicted logits from the model.
        threshold (float): Threshold to convert probabilities to binary predictions.
        is_multilabel (bool): Whether the task is multi-label classification.

    Returns:
        dict: Dictionary containing accuracy, precision, recall, and F1 score.
        np.ndarray: Confusion matrix.
    """
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predicted_logits)).numpy()

    if is_multilabel:
        # For multi-label, apply threshold to each label independently
        predicted_labels = (probs >= threshold).astype(int)
    else:
        # For binary (hatespeech), apply threshold to the single output
        predicted_labels = (probs >= threshold).astype(int)


    # Calculate metrics
    accuracy = accuracy_score(true_labels.flatten(), predicted_labels.flatten())
    balanced_accuracy = balanced_accuracy_score(true_labels.flatten(), predicted_labels.flatten())
    precision = precision_score(true_labels.flatten(), predicted_labels.flatten(), average='micro', zero_division=0)
    recall = recall_score(true_labels.flatten(), predicted_labels.flatten(), average='micro', zero_division=0)
    f1 = f1_score(true_labels.flatten(), predicted_labels.flatten(), average='micro', zero_division=0)

    # Calculate confusion matrix - Note: For multi-label, CM is often calculated per label or overall
    # A simple overall CM for multilabel can be misleading. We'll calculate it based on flattened arrays
    # which treats each label prediction as a separate instance.
    cm = confusion_matrix(true_labels.flatten(), predicted_labels.flatten())


    metrics = {
        'accuracy': accuracy,
        'balanced_accuracy': balanced_accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    return metrics, cm

# Calculate metrics and confusion matrices for each group

# Hatespeech (Binary Classification)
hatespeech_metrics, hatespeech_cm = calculate_metrics_and_confusion_matrix(
    true_hatespeech, predicted_hatespeech_logits, is_multilabel=False
)
print("Hatespeech Metrics:", hatespeech_metrics)
print("Hatespeech Confusion Matrix:\n", hatespeech_cm)


# Sentiment (Multi-label Classification)
sentiment_metrics, sentiment_cm = calculate_metrics_and_confusion_matrix(
    true_sentiment_labels, predicted_sentiment_logits, is_multilabel=True
)
print("\nSentiment Metrics (Micro Avg):", sentiment_metrics)
print("Sentiment Confusion Matrix (Flattened):\n", sentiment_cm)


# Directness (Multi-label Classification - treated as binary for each category)
directness_metrics, directness_cm = calculate_metrics_and_confusion_matrix(
    true_directness_labels, predicted_directness_logits, is_multilabel=True
)
print("\nDirectness Metrics (Micro Avg):", directness_metrics)
print("Directness Confusion Matrix (Flattened):\n", directness_cm)

# Target (Multi-label Classification - treated as binary for each category)
target_metrics, target_cm = calculate_metrics_and_confusion_matrix(
    true_target_labels, predicted_target_logits, is_multilabel=True
)
print("\nTarget Metrics (Micro Avg):", target_metrics)
print("Target Confusion Matrix (Flattened):\n", target_cm)

# Group (Multi-label Classification - treated as binary for each category)
group_metrics, group_cm = calculate_metrics_and_confusion_matrix(
    true_group_labels, predicted_group_logits, is_multilabel=True
)
print("\nGroup Metrics (Micro Avg):", group_metrics)
print("Group Confusion Matrix (Flattened):\n", group_cm)

In [ ]:
# Visualize Flattened Confusion Matrices for each group

# Hatespeech Flattened Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(hatespeech_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Hatespeech')
plt.show()

# Sentiment Flattened Confusion Matrix (Micro Avg across labels)
# Note: This matrix is based on the flattened true and predicted labels across all sentiment types.
plt.figure(figsize=(6, 4))
sns.heatmap(sentiment_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Sentiment (Micro Avg)')
plt.show()

# Directness Flattened Confusion Matrix (Micro Avg across categories)
# Note: This matrix is based on the flattened true and predicted labels across all directness categories.
plt.figure(figsize=(6, 4))
sns.heatmap(directness_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Directness (Micro Avg)')
plt.show()


# Target Flattened Confusion Matrix (Micro Avg across categories)
# Note: This matrix is based on the flattened true and predicted labels across all target categories.
plt.figure(figsize=(6, 4))
sns.heatmap(target_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Target (Micro Avg)')
plt.show()

# Group Flattened Confusion Matrix (Micro Avg across categories)
# Note: This matrix is based on the flattened true and predicted labels across all group categories.
plt.figure(figsize=(6, 4))
sns.heatmap(group_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Group (Micro Avg)')
plt.show()

In [ ]:
# Visualize Confusion Matrices for each individual binary label

# Apply sigmoid to get probabilities
sigmoid = torch.nn.Sigmoid()
predicted_probs_np = sigmoid(torch.Tensor(predicted_logits_np)).numpy()

# Apply threshold to get binary predictions for each label
predicted_binary_labels_np = (predicted_probs_np >= 0.5).astype(int)


# Iterate through each binary label and plot its confusion matrix
for i, label_name in id2label.items():
    true_binary = true_labels_np[:, i]
    predicted_binary = predicted_binary_labels_np[:, i]

    # Calculate confusion matrix for the current binary label
    cm = confusion_matrix(true_binary, predicted_binary)

    # Plot the confusion matrix
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix for: {label_name}')
    plt.show()

## UnFrozen Multi-Label Prediction

In [ ]:
stop

In [1]:
!rm -r "google/rembert" # if the next cell gives error "not a string", run this cell

rm: cannot remove 'google/rembert': No such file or directory


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('google/rembert')
model = AutoModelForSequenceClassification.from_pretrained('google/rembert',
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(all_binary_labels),
                                                           id2label=id2label,
                                                           label2id=label2id)
model.gradient_checkpointing_enable() # to help memory issues

NameError: name 'id2label' is not defined

In [27]:
import torch
torch.cuda.empty_cache()

In [28]:
for name, param in model.named_parameters():
    print(name)

rembert.embeddings.word_embeddings.weight
rembert.embeddings.position_embeddings.weight
rembert.embeddings.token_type_embeddings.weight
rembert.embeddings.LayerNorm.weight
rembert.embeddings.LayerNorm.bias
rembert.encoder.embedding_hidden_mapping_in.weight
rembert.encoder.embedding_hidden_mapping_in.bias
rembert.encoder.layer.0.attention.self.query.weight
rembert.encoder.layer.0.attention.self.query.bias
rembert.encoder.layer.0.attention.self.key.weight
rembert.encoder.layer.0.attention.self.key.bias
rembert.encoder.layer.0.attention.self.value.weight
rembert.encoder.layer.0.attention.self.value.bias
rembert.encoder.layer.0.attention.output.dense.weight
rembert.encoder.layer.0.attention.output.dense.bias
rembert.encoder.layer.0.attention.output.LayerNorm.weight
rembert.encoder.layer.0.attention.output.LayerNorm.bias
rembert.encoder.layer.0.intermediate.dense.weight
rembert.encoder.layer.0.intermediate.dense.bias
rembert.encoder.layer.0.output.dense.weight
rembert.encoder.layer.0.output

In [29]:
# Freeze all of the pre-trained BERT layers to make the fine tuning go much faster.
# Then later we'll try unfreezing some or all layers and see what works better.
# We need to keep the final classification layer unfrozen no matter what,
# because that's a new layer that hasn't been trained at all yet, and needs to be trained for our task.


for name, param in model.named_parameters():
    if name.split(".")[0] == "rembert":
        param.requires_grad = True

In [30]:
summary(model)

Layer (type:depth-idx)                                       Param #
RemBertForSequenceClassification                             --
├─RemBertModel: 1-1                                          --
│    └─RemBertEmbeddings: 2-1                                --
│    │    └─Embedding: 3-1                                   64,076,800
│    │    └─Embedding: 3-2                                   131,072
│    │    └─Embedding: 3-3                                   512
│    │    └─LayerNorm: 3-4                                   512
│    │    └─Dropout: 3-5                                     --
│    └─RemBertEncoder: 2-2                                   --
│    │    └─Linear: 3-6                                      296,064
│    │    └─ModuleList: 3-7                                  510,087,168
│    └─RemBertPooler: 2-3                                    --
│    │    └─Linear: 3-8                                      1,328,256
│    │    └─Tanh: 3-9                                        --

In [31]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

batch_size = 64
metric_name = "eval_f1" # Changed to monitor a specific per-label F1

args = TrainingArguments(
    f"google/rembert",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, # to help memory management
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    # Use the micro-averaged F1 as the metric to monitor
    metric_for_best_model=metric_name,
    greater_is_better=True, # F1 score is better when higher
    report_to='none', # Disable reporting to wandb for this example

    # try to fix no log issue (for loss)
    # KEY for logging:
    logging_strategy="steps",         # or "epoch"
    logging_steps=10,                 # log every 10 steps
    disable_tqdm=False,
)

# Add EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3, # Number of evaluation epochs to wait before stopping
    early_stopping_threshold=0.0 # Minimum improvement to consider as progress
)

In [32]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, balanced_accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np # Import numpy

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels.astype(int) # Ensure labels are integers
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true.flatten(), y_pred.flatten())
    precision = precision_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    recall = recall_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'balanced_accuracy': balanced_accuracy,
               'precision': precision,
               'recall': recall}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions

    # Ensure labels are a 2D numpy array of integers
    labels_np = np.array(p.label_ids)
    if labels_np.ndim == 1:
        # This might happen if the batch size is 1, reshape to (1, num_labels)
        labels_np = labels_np.reshape(1, -1)
    elif labels_np.ndim > 2:
        # Handle cases where labels might have extra dimensions unexpectedly
        # This might require more specific reshaping based on the actual structure
        # For now, let's assume the expected shape is (batch_size, num_labels)
        # and try to flatten and reshape if necessary.
        # A safer approach might be to raise an error or log a warning here.
        print(f"Warning: Unexpected label dimension: {labels_np.ndim}")
        # Attempt to flatten and reshape - adjust if your data structure is different
        labels_np = labels_np.reshape(labels_np.shape[0], -1)


    result = multi_label_metrics(
        predictions=preds,
        labels=labels_np)
    return result

In [33]:
#forward pass
# # Check the type of the data after setting the format
# print(f"Type of input_ids[0]: {encoded_dataset['train']['input_ids'][0].type()}")
# print(f"Type of labels[0]: {encoded_dataset['train'][0]['hatespeech_labels'].type()}")

# Also print a sample example to see the structure
print("Sample example after setting format:")
print(encoded_dataset['train'][0])

# Manually get a sample and prepare input for the model
sample = encoded_dataset['train'][0]

# Convert lists to PyTorch tensors before unsqueezing
input_ids_tensor = torch.tensor(sample['input_ids']).unsqueeze(0)
attention_mask_tensor = torch.tensor(sample['attention_mask']).unsqueeze(0)
labels_tensor = torch.tensor(sample['labels']).unsqueeze(0)

# Move tensors to the same device as the model
device = model.device
input_ids_tensor = input_ids_tensor.to(device)
attention_mask_tensor = attention_mask_tensor.to(device)
labels_tensor = labels_tensor.to(device)


outputs = model(input_ids=input_ids_tensor, attention_mask=attention_mask_tensor, labels=labels_tensor)
outputs

Sample example after setting format:
{'input_ids': [312, 573, 8671, 114741, 573, 81901, 2262, 14900, 573, 43540, 3902, 313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

SequenceClassifierOutput(loss=tensor(0.7258, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.1046,  0.3788,  0.5882, -0.1440,  0.0240,  0.0198, -0.2708,  0.3418,
          0.1261,  0.3965, -0.0056,  0.0007, -0.1076,  0.1516, -0.2907, -0.0205,
         -0.1386,  0.0560, -0.1560, -0.0673, -0.1001, -0.3424,  0.3521,  0.2904,
          0.2584, -0.0036, -0.5292,  0.1168,  0.3695, -0.0776, -0.5303,  0.0545,
          0.0719,  0.1498,  0.0736, -0.1479]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-34-3132099784.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [35]:
trainer.can_return_loss = True # try to fix no log issue (for loss)

In [36]:
print(encoded_dataset['train']['labels'][13000])

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [37]:
# import torch
# import gc

# gc.collect()                 # Collects Python garbage (old objects)
# torch.cuda.empty_cache()     # Releases unused memory from GPU

In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 44.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 35.38 MiB is free. Process 15252 has 22.12 GiB memory in use. Of the allocated memory 20.68 GiB is allocated by PyTorch, and 1.21 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.evaluate()

In [ ]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
logits = outputs.logits
logits.shape

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print("Predicted labels (names):", predicted_labels)
print("Predicted labels (binary values):", predictions)

In [ ]:
true_labels = []
predicted_logits = []

device = trainer.model.device

for sample in encoded_dataset["validation"]:
    inputs = {
        'input_ids': torch.tensor([sample['input_ids']]).to(device),
        'attention_mask': torch.tensor([sample['attention_mask']]).to(device)
    }

    with torch.no_grad():
        outputs = trainer.model(**inputs)

    # Append the true labels and predicted logits
    true_labels.append(sample['labels'])
    predicted_logits.append(outputs.logits.squeeze().cpu().numpy())


In [ ]:
# Convert lists to numpy arrays
true_labels_np = np.array(true_labels)
predicted_logits_np = np.array(predicted_logits)

# Define indices for each label group based on all_binary_labels
hatespeech_idx = all_binary_labels.index('hatespeech')
sentiment_indices = [all_binary_labels.index(label) for label in binary_label_columns if 'sentiment' in label]
directness_indices = [all_binary_labels.index(label) for label in directness_binary_labels]
target_indices = [all_binary_labels.index(label) for label in target_binary_labels]
group_indices = [all_binary_labels.index(label) for label in group_binary_labels]


# Split true labels and predicted logits
true_hatespeech = true_labels_np[:, hatespeech_idx]
predicted_hatespeech_logits = predicted_logits_np[:, hatespeech_idx]

true_sentiment_labels = true_labels_np[:, sentiment_indices]
predicted_sentiment_logits = predicted_logits_np[:, sentiment_indices]

true_directness_labels = true_labels_np[:, directness_indices]
predicted_directness_logits = predicted_logits_np[:, directness_indices]

true_target_labels = true_labels_np[:, target_indices]
predicted_target_logits = predicted_logits_np[:, target_indices]

true_group_labels = true_labels_np[:, group_indices]
predicted_group_logits = predicted_logits_np[:, group_indices]

print("True Hatespeech Labels shape:", true_hatespeech.shape)
print("Predicted Hatespeech Logits shape:", predicted_hatespeech_logits.shape)
print("True Sentiment Labels shape:", true_sentiment_labels.shape)
print("Predicted Sentiment Logits shape:", predicted_sentiment_logits.shape)
print("True Directness Labels shape:", true_directness_labels.shape)
print("Predicted Directness Logits shape:", predicted_directness_logits.shape)
print("True Target Labels shape:", true_target_labels.shape)
print("Predicted Target Logits shape:", predicted_target_logits.shape)
print("True Group Labels shape:", true_group_labels.shape)
print("Predicted Group Logits shape:", predicted_group_logits.shape)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_metrics_and_confusion_matrix(true_labels, predicted_logits, threshold=0.5, is_multilabel=True):
    """
    Calculates classification metrics and confusion matrix.

    Args:
        true_labels (np.ndarray): True labels (binary).
        predicted_logits (np.ndarray): Predicted logits from the model.
        threshold (float): Threshold to convert probabilities to binary predictions.
        is_multilabel (bool): Whether the task is multi-label classification.

    Returns:
        dict: Dictionary containing accuracy, precision, recall, and F1 score.
        np.ndarray: Confusion matrix.
    """
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predicted_logits)).numpy()

    if is_multilabel:
        # For multi-label, apply threshold to each label independently
        predicted_labels = (probs >= threshold).astype(int)
    else:
        # For binary (hatespeech), apply threshold to the single output
        predicted_labels = (probs >= threshold).astype(int)


    # Calculate metrics
    accuracy = accuracy_score(true_labels.flatten(), predicted_labels.flatten())
    balanced_accuracy = balanced_accuracy_score(true_labels.flatten(), predicted_labels.flatten())
    precision = precision_score(true_labels.flatten(), predicted_labels.flatten(), average='micro', zero_division=0)
    recall = recall_score(true_labels.flatten(), predicted_labels.flatten(), average='micro', zero_division=0)
    f1 = f1_score(true_labels.flatten(), predicted_labels.flatten(), average='micro', zero_division=0)

    # Calculate confusion matrix - Note: For multi-label, CM is often calculated per label or overall
    # A simple overall CM for multilabel can be misleading. We'll calculate it based on flattened arrays
    # which treats each label prediction as a separate instance.
    cm = confusion_matrix(true_labels.flatten(), predicted_labels.flatten())


    metrics = {
        'accuracy': accuracy,
        'balanced_accuracy': balanced_accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    return metrics, cm

# Calculate metrics and confusion matrices for each group

# Hatespeech (Binary Classification)
hatespeech_metrics, hatespeech_cm = calculate_metrics_and_confusion_matrix(
    true_hatespeech, predicted_hatespeech_logits, is_multilabel=False
)
print("Hatespeech Metrics:", hatespeech_metrics)
print("Hatespeech Confusion Matrix:\n", hatespeech_cm)


# Sentiment (Multi-label Classification)
sentiment_metrics, sentiment_cm = calculate_metrics_and_confusion_matrix(
    true_sentiment_labels, predicted_sentiment_logits, is_multilabel=True
)
print("\nSentiment Metrics (Micro Avg):", sentiment_metrics)
print("Sentiment Confusion Matrix (Flattened):\n", sentiment_cm)


# Directness (Multi-label Classification - treated as binary for each category)
directness_metrics, directness_cm = calculate_metrics_and_confusion_matrix(
    true_directness_labels, predicted_directness_logits, is_multilabel=True
)
print("\nDirectness Metrics (Micro Avg):", directness_metrics)
print("Directness Confusion Matrix (Flattened):\n", directness_cm)

# Target (Multi-label Classification - treated as binary for each category)
target_metrics, target_cm = calculate_metrics_and_confusion_matrix(
    true_target_labels, predicted_target_logits, is_multilabel=True
)
print("\nTarget Metrics (Micro Avg):", target_metrics)
print("Target Confusion Matrix (Flattened):\n", target_cm)

# Group (Multi-label Classification - treated as binary for each category)
group_metrics, group_cm = calculate_metrics_and_confusion_matrix(
    true_group_labels, predicted_group_logits, is_multilabel=True
)
print("\nGroup Metrics (Micro Avg):", group_metrics)
print("Group Confusion Matrix (Flattened):\n", group_cm)

In [ ]:
# Visualize Flattened Confusion Matrices for each group

# Hatespeech Flattened Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(hatespeech_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Hatespeech')
plt.show()

# Sentiment Flattened Confusion Matrix (Micro Avg across labels)
# Note: This matrix is based on the flattened true and predicted labels across all sentiment types.
plt.figure(figsize=(6, 4))
sns.heatmap(sentiment_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Sentiment (Micro Avg)')
plt.show()

# Directness Flattened Confusion Matrix (Micro Avg across categories)
# Note: This matrix is based on the flattened true and predicted labels across all directness categories.
plt.figure(figsize=(6, 4))
sns.heatmap(directness_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Directness (Micro Avg)')
plt.show()


# Target Flattened Confusion Matrix (Micro Avg across categories)
# Note: This matrix is based on the flattened true and predicted labels across all target categories.
plt.figure(figsize=(6, 4))
sns.heatmap(target_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Target (Micro Avg)')
plt.show()

# Group Flattened Confusion Matrix (Micro Avg across categories)
# Note: This matrix is based on the flattened true and predicted labels across all group categories.
plt.figure(figsize=(6, 4))
sns.heatmap(group_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Flattened Confusion Matrix for: Group (Micro Avg)')
plt.show()

In [ ]:
# Visualize Confusion Matrices for each individual binary label

# Apply sigmoid to get probabilities
sigmoid = torch.nn.Sigmoid()
predicted_probs_np = sigmoid(torch.Tensor(predicted_logits_np)).numpy()

# Apply threshold to get binary predictions for each label
predicted_binary_labels_np = (predicted_probs_np >= 0.5).astype(int)


# Iterate through each binary label and plot its confusion matrix
for i, label_name in id2label.items():
    true_binary = true_labels_np[:, i]
    predicted_binary = predicted_binary_labels_np[:, i]

    # Calculate confusion matrix for the current binary label
    cm = confusion_matrix(true_binary, predicted_binary)

    # Plot the confusion matrix
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['True 0', 'True 1'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix for: {label_name}')
    plt.show()